<a href="https://colab.research.google.com/github/lleo022/image_recognition_cnn/blob/main/Image_Recognition_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import torch & load onto GPU
import torch

if torch.cuda.is_available():
  device = torch.device('cuda')
  print('Running on GPU')
  print(torch.cuda.get_device_name(0))
else:
  device = torch.device('cpu')
  print('Running on CPU')

Running on GPU
Tesla T4


In [ ]:
class CustomDataset(torch.utils.data.Dataset):
  def __init__(self, images, labels):
    super(CustomDataset, self).__init__()
    self.images = images
    self.labels = labels

  def __len__(self):
    return len(self.images)

  def __getitem__(self, i):
    return (self.images[i], self.labels[i])

images = [1, 2, 3, 7, 8, 9, 10, 12, 15]
labels = [4, 5, 6, 15, 25, 37, 80, 78, 1]
trainData = CustomDataset(images, labels)

BATCH_SIZE = 1
trainLoader = torch.utils.data.DataLoader(trainData, batch_size = BATCH_SIZE, shuffle = True)

next(iter(trainLoader))


[tensor([8]), tensor([25])]

In [ ]:
# load torchvision CIFAR10
import torchvision

train_dataset = torchvision.datasets.CIFAR10(root='./cifar10', transform=torchvision.transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./cifar10', train=False, transform=torchvision.transforms.ToTensor(), download=True)

100%|██████████| 170M/170M [00:05<00:00, 33.6MB/s]


In [ ]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self, channels, kernels, paddings, strides, poolings, linears):
        super(CNN, self).__init__()

        self.layer1 = nn.Linear(567, 234)
        self.activation1 = nn.ReLU()
        # other activation functions: nn.Softmax, nn.Sigmoid
        self.layer2 = nn.Linear(234, 128)
        self.activation2 = nn.RelU()
        self.layer3 = nn.Linear(128, 10)

        channels = [3] + channels
        size_after_each = 32
        # stores height/width after each convolution

        self.conv_layers = []
        for i in range(len(channels)-1):
          size_after_each = (size_after_each - kernels[i] + 2*paddings[i])/strides[i] + 1
          self.conv_layers.append(
              nn.Sequential(
                  nn.Conv2d(channels[i], channels[i+1], kernel_size=kernels[i],
                            padding=paddings[i], stride=strides[i]),
                  nn.MaxPool2d(poolings[i], poolings[i]),
                  nn.BatchNorm2d(channels[i+1]),
                  nn.ReLU(),
                  nn.Dropout(0.05)
              )
          )
        self.conv_layers = nn.Sequential(self.conv_layers)

        self.flattener = nn.Flatten()

        self.fc_layers = []
        linears = [channels[-1]*size_after_each*size_after_each] + 1
        for i in range(len(linears)-1):
          self.fc_layers.append(
              nn.Sequential(
                  nn.Linear(linears[i], linears[i+1]),
                  nn.ReLU(),
                  nn.Dropout(0.05)
              )
          )

        self.final_layer = nn.Linear(linears[-1], 10)

        self.softmax = nn.Softmax()

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.flattener(x)
        x = self.fc_layers(x)
        x = self.final_layer(x)
        return self.softmax(x)

#channels, kernels, paddings, strides, poolings
model = CNN().to(device)

